# How to Add Ad-hoc Tool Calling Capability to LLMs and Chat Models

**⚠️ Caution**: Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the how to use a chat model to call tools guide for more information.

## Prerequisites
This guide assumes familiarity with the following concepts:
- LangChain Tools
- Function/tool calling
- Chat models
- LLMs

In this guide, we'll see how to add ad-hoc tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools.


In [ ]:
# Install required packages
%pip install --upgrade --quiet langchain langchain-community


In [ ]:
# Optional: LangSmith setup
import getpass
import os

# Uncomment the lines below if you'd like to use LangSmith:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


## Model Selection

You can select any of the given models for this how-to guide. Keep in mind that most of these models already support native tool calling, so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the how to use a chat model to call tools guide.

Available options: OpenAI, Anthropic, Azure, Google, Cohere, NVIDIA, FireworksAI, Groq, MistralAI, TogetherAI

### Option 1: Using TogetherAI with Mixtral (Example)


In [ ]:
# Example: Using TogetherAI with Mixtral
# pip install -qU langchain-openai (uncomment to install)

# import getpass
# import os
# 
# os.environ["TOGETHER_API_KEY"] = getpass.getpass()
# 
# from langchain_openai import ChatOpenAI
# 
# model = ChatOpenAI(
#     base_url="https://api.together.xyz/v1",
#     api_key=os.environ["TOGETHER_API_KEY"],
#     model="mistralai/Mixtral-8x7B-Instruct-v0.1",
# )


### Option 2: Using Ollama with phi3 (Recommended for this example)

To illustrate the idea, we'll use phi3 via Ollama, which does NOT have native support for tool calling. If you'd like to use Ollama as well follow the Ollama installation instructions.


In [ ]:
# Using Ollama with phi3 (does NOT have native tool calling support)
from langchain_community.llms import Ollama

model = Ollama(model="phi3")


In [ ]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)


In [ ]:
# Test the multiply tool
multiply.invoke({"x": 4, "y": 5})


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)


In [ ]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)


In [ ]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})


In [ ]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)


In [ ]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})


In [ ]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})
